In [9]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import h5py as h5
import os
import fftx
import h5py

In [2]:
import wfnreader 
import symmetry_maps
wfn = wfnreader.WFNReader("WFN.h5")
wfn_unf = wfnreader.WFNReader("WFN_unfolded.h5")
sym = symmetry_maps.SymMaps(wfn)
print(sym.irk_to_k_map)
print(sym.irk_sym_map)
print(sym.kpoint_map)
print(sym.kpoint_map_ibz_ids)
#print(sym.kfull_symmap)

[0 1 2 2 3 1 1 2 3]
[0 0 0 3 0 2 3 2 1]
[0 0 0 2 0 3 2 3 1]
[0 0 0 2 0 3 2 3 1]


In [18]:
sym.get_gvecs_kfull(wfn,8)

array([[-1,  0,  0],
       [-1,  0,  1],
       [-1,  0, -1],
       ...,
       [ 3, -7, -2],
       [ 2, -7,  2],
       [ 2, -7, -2]], shape=(3576, 3), dtype=int32)

In [10]:
h5py.File('eps0mat.h5','r')['mats/matrix'].shape

(1, 1, 1, 303, 303, 2)

In [5]:
25*25*100

62500

In [ ]:
fftbox = np.zeros((2,*wfn.fft_grid),dtype=np.complex128)
sym.get_cnk_fullzone_to_box(wfn,25,5,fftbox)



In [4]:

for i in range(9):
    print(" ")
    print(wfn.kpoints[sym.irk_to_k_map[i]])
    print((sym.sym_mats_k[sym.irk_sym_map[i]] @ wfn.kpoints[sym.irk_to_k_map[i]])%1.0)
    print(sym.unfolded_kpts[i])


 
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
 
[0.         0.33333333 0.        ]
[0.         0.33333333 0.        ]
[0.         0.33333333 0.        ]
 
[0.         0.66666667 0.        ]
[0.         0.66666667 0.        ]
[0.         0.66666667 0.        ]
 
[0.         0.66666667 0.        ]
[0.33333333 0.         0.        ]
[0.33333333 0.         0.        ]
 
[0.33333333 0.33333333 0.        ]
[0.33333333 0.33333333 0.        ]
[0.33333333 0.33333333 0.        ]
 
[0.         0.33333333 0.        ]
[0.33333333 0.66666667 0.        ]
[0.33333333 0.66666667 0.        ]
 
[0.         0.33333333 0.        ]
[0.66666667 0.         0.        ]
[0.66666667 0.         0.        ]
 
[0.         0.66666667 0.        ]
[0.66666667 0.33333333 0.        ]
[0.66666667 0.33333333 0.        ]
 
[0.33333333 0.33333333 0.        ]
[0.66666667 0.66666667 0.        ]
[0.66666667 0.66666667 0.        ]


In [93]:
# psi_kbar from OG wfn file
print(wfn.kpoints[2])
wfn_kbarG = wfn.get_cnk(2,26)
kbar_gvecs = wfn.get_gvec_nk(2)
wfn_kbarGgrid = np.zeros((2,*wfn.fft_grid),dtype=np.complex128)
for ispin in range(2):
    wfn_kbarGgrid[ispin,kbar_gvecs[:,0],kbar_gvecs[:,1],kbar_gvecs[:,2]] = wfn_kbarG[ispin]

# psi_k unfolded from auxiliary file
print(wfn_unf.kpoints[4])
wfn_kG = wfn_unf.get_cnk(4,26)
k_gvecs = wfn_unf.get_gvec_nk(4)
# Sinv[8] @ kbar = k - Gs
Gkk = (wfn.kpoints[2] - sym_list[3] @ wfn_unf.kpoints[4]).astype(int)
print(Gkk)
k_gvecs_rot = np.einsum('ij,kj->ki', sym_list[3], k_gvecs) # kgrid.x says sym 9 maps k to kbar
#k_gvecs_rot += Gkk

#wfn_kG = np.einsum('jk,kl->jl', sym.U_spinor[8], wfn_kG)
wfn_kGgrid = np.zeros((2,*wfn.fft_grid),dtype=np.complex128)
for ispin in range(2):
    wfn_kGgrid[ispin,k_gvecs_rot[:,0],k_gvecs_rot[:,1],k_gvecs_rot[:,2]] = wfn_kG[ispin]

[0.         0.66666667 0.        ]
[3.33333333e-01 6.11095340e-17 0.00000000e+00]
[0 1 0]


In [59]:
print(np.abs(wfn_kbarGgrid[:,0,0,0]))
print(np.abs(wfn_kGgrid[:,0,0,0]))



[0.04894177 0.04910857]
[6.93272778e-02 1.91431198e-06]


In [5]:
def check_vectors_equivalent(arr1, arr2, tol=1e-4):
    """
    Check if two arrays contain the same vectors up to reordering and numerical precision.
    
    Args:
        arr1: numpy array of shape (n,3)
        arr2: numpy array of shape (n,3)
        tol: tolerance for numerical comparison (default 1e-8)
    
    Returns:
        bool: True if arrays contain same vectors, False otherwise
    """
    if arr1.shape != arr2.shape:
        return False
    
    # Compute all pairwise differences
    diffs = np.abs(arr1[:, None] - arr2)  # Shape: (n,n,3)
    # Sum over vector components
    diffs = np.sum(diffs, axis=2)  # Shape: (n,n)
    # Find minimum difference for each vector in arr1
    min_diffs = np.min(diffs, axis=1)
    
    # Check if all vectors in arr1 have a match in arr2
    return np.all(min_diffs < tol)

# Example usage:
# vectors1 = np.array([[1,0,0], [0,1,0], [0,0,1]])
# vectors2 = np.array([[0,0,1], [1,0,0], [0,1,0]])
# are_equal = check_vectors_equivalent(vectors1, vectors2)

In [8]:
np.set_printoptions(suppress=True,precision=4)
for i in range(12):
    pos1 = np.einsum('ij,kj->ki',wfn.sym_matrices[i],wfn.atom_crys)%1.0
    pos2 = wfn.atom_crys%1.0
    print(check_vectors_equivalent(pos1,pos2))

True
True
True
True
True
True
True
True
True
True
True
True


In [136]:
np.einsum('ij,kj->ki',wfn.sym_matrices[6],wfn.atom_crys) - wfn.atom_crys

array([[ 0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    , -0.2639],
       [ 0.    ,  0.    ,  0.2639]])

In [9]:
print(wfn_unf.kpoints[3])
for i in range(12):
    print((wfn.sym_matrices[i].T @ wfn_unf.kpoints[3]) % 1.0)
print(wfn_unf.kpoints[5])

[0.3333 0.3333 0.    ]
[0.3333 0.3333 0.    ]
[0.6667 0.6667 0.    ]
[0.3333 0.3333 0.    ]
[0.3333 0.3333 0.    ]
[0.6667 0.6667 0.    ]
[0.6667 0.6667 0.    ]
[0.3333 0.3333 0.    ]
[0.6667 0.6667 0.    ]
[0.3333 0.3333 0.    ]
[0.3333 0.3333 0.    ]
[0.6667 0.6667 0.    ]
[0.6667 0.6667 0.    ]
[0.6667 0.6667 0.    ]


In [29]:
def unfold_wfnkbar_by_sym(wfn,sym_mat,kbar_id,k_id,band_id):
    # sym_id: index of sym in wfn.sym_matrices
    # kbar_id: index of kbar in wfn.kpoints FULL ZONE ID
    # k_id: index of k in wfn.kpoints (should eventually be in sym.unfolded_kpts)
    # band_id: index of band to rotate, should eventually do all at that kpt
    # relationship: u_n({S|v}.T kbar) (G) = u_nkbar({S|v}.T G - G_S), apparently..
    wfn_kG = wfn.get_cnk(kbar_id,band_id)
    k_gvecs = wfn.get_gvec_nk(kbar_id)
    sym_krep = sym_mat # note we apply with no changes
    
    q_full = sym_krep @ wfn.kpoints[kbar_id] #+ 0.001
    q_inzone = q_full%1.0
    q_inzone[q_inzone>0.9999] = 0.0
    Gkk = q_inzone - q_full
    #print(f'Gkk, should be integer only: {Gkk}')
    Gkk = Gkk.astype(int) 
    k_gvecs_rot = np.einsum('ij,kj->ki', sym_krep.astype(np.int32), k_gvecs) # kgrid.x says sym 9 maps k to kbar
    k_gvecs_rot -= Gkk
    #wfn_kG = np.einsum('jk,kl->jl', sym.U_spinor[sym_id], wfn_kG)
    wfn_kGgrid = np.zeros((2,*wfn.fft_grid),dtype=np.complex128)
    for ispin in range(2):
        wfn_kGgrid[ispin,k_gvecs_rot[:,0],k_gvecs_rot[:,1],k_gvecs_rot[:,2]] = wfn_kG[ispin]
    return wfn_kGgrid


# sym convention: in kgrid.log, sym (isym) maps kpt (ikbar) to kpt (ik). end of story
# problem being that the syms in kgrid.log are scrambled relative to the syms in WFN.h5

In [30]:
# kgrid.log: kpt 5 (0.67,0.67,0) maps to kpt 3 (0.33,0.33,0) by sym 2
wfn_k5og = wfn_unf.get_cnk(5,25)
wfn_k5gvec = wfn_unf.get_gvec_nk(5)
wfn_k5oggrid = np.zeros((2,*wfn_unf.fft_grid),dtype=np.complex128)
for ispin in range(2):
    wfn_k5oggrid[ispin,wfn_k5gvec[:,0],wfn_k5gvec[:,1],wfn_k5gvec[:,2]] = wfn_k5og[ispin]


In [31]:
# only k values in this little group should be (0.33,0.33,0) and (0.67,0.67,0)

for i in range(12):
    sym_mat = wfn.sym_matrices[i].T
    wfn_k5unf = unfold_wfnkbar_by_sym(wfn_unf,sym_mat,3,5,25)
    print(f"Sk: {(sym_mat @ wfn_unf.kpoints[3]) % 1.0}, c_nSk(1:4): {np.abs(wfn_k5unf[1,0,:4,0])+np.abs(wfn_k5unf[0,0,:4,0])}")
print(f"true Sk: {wfn_unf.kpoints[5]}, true c_nSk {np.abs(wfn_k5oggrid[0,0,:4,0])+np.abs(wfn_k5oggrid[1,0,:4,0])}")

Sk: [0.3333 0.3333 0.    ], c_nSk(1:4): [0.022  0.0817 0.0231 0.0048]
Sk: [0.6667 0.6667 0.    ], c_nSk(1:4): [0.2063 0.0335 0.0114 0.0014]
Sk: [0.3333 0.3333 0.    ], c_nSk(1:4): [0.022  0.0817 0.0231 0.0048]
Sk: [0.3333 0.3333 0.    ], c_nSk(1:4): [0.022  0.0817 0.0231 0.0048]
Sk: [0.6667 0.6667 0.    ], c_nSk(1:4): [0.2063 0.0335 0.0114 0.0014]
Sk: [0.6667 0.6667 0.    ], c_nSk(1:4): [0.2063 0.0335 0.0114 0.0014]
Sk: [0.3333 0.3333 0.    ], c_nSk(1:4): [0.022  0.0817 0.0231 0.0048]
Sk: [0.6667 0.6667 0.    ], c_nSk(1:4): [0.2063 0.0335 0.0114 0.0014]
Sk: [0.3333 0.3333 0.    ], c_nSk(1:4): [0.022  0.0817 0.0231 0.0048]
Sk: [0.3333 0.3333 0.    ], c_nSk(1:4): [0.022  0.0817 0.0231 0.0048]
Sk: [0.6667 0.6667 0.    ], c_nSk(1:4): [0.2063 0.0335 0.0114 0.0014]
Sk: [0.6667 0.6667 0.    ], c_nSk(1:4): [0.2063 0.0335 0.0114 0.0014]
true Sk: [0.6667 0.6667 0.    ], true c_nSk [0.2063 0.0335 0.0114 0.0014]
